In [1]:
"""
Testar olika metoder för metamodell learn och gör en gridsearch på metamodeller
"""

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix

from IPython.display import display 
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.width', 260)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 80)
from category_encoders import TargetEncoder
from category_encoders import CatBoostEncoder
from sklearn.model_selection import TimeSeriesSplit

import pickle
# import concurrent.futures
import sys

sys.path.append(
    'C:\\Users\\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel\\')
pref = '../'
import travdata as td

# sys.path.append('C:\\Users\\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel\\modeller\\')
import V75_scraping as vs

import typ as tp



END 13.43.23


In [3]:
# Create a dummy list with strings
l =      ['abc', 'bdd', 'csn', 'david', 'edward', 'fridz', 'gustav', 'hans', 'ingvar']
print([item for item in l if 'ar' in item])


['edward', 'ingvar']
['abc', 'bdd', 'csn', 'david', 'edward', 'fridz', 'gustav', 'hans', 'ingvar']


# Kollar XGBoost

In [2]:
def lägg_in_antal_hästar(df_):
    df = df_.copy()
    df['ant_per_lopp'] = None
    df['ant_per_lopp'] = df.groupby(['datum', 'avd'])['avd'].transform('count')
    return df


def lägg_in_diff_motståndare(X_, ant_motståndare):
    X = X_.copy()

    # set X['motståndare1'] to largest streck in every avd
    grouped = X.groupby(['datum', 'avd'])['streck']
    X['diff1'] = grouped.transform(max) - X.streck

    for i in range(2, ant_motståndare+1):
        # set X['motståndare'+str(i)] to ith largest streck in every avd
        X['diff' +
            str(i)] = grouped.transform(lambda x: x.nlargest(i).min()) - X.streck

    return X

def prepare_for_model(X_):
    X = X_.copy()
    X = lägg_in_antal_hästar(X)
    X = lägg_in_diff_motståndare(X, 3)


Skapar feature-filerna (en gång för alla)

In [3]:
if False:     # Kör bara vid behov av att skapa nya filer med features
    # get travdata
    v75 = td.v75(pref='../')
    df, _ = v75.förbered_data(extra=True)

    # extract categorical features
    cat_features = df.select_dtypes(include=['object']).columns
    # extract all other features
    num_features = df.select_dtypes(exclude=['object']).columns

    L1_features = ['cat1_L1', 'cat2_L1', 'xgb1_L1', 'xgb1_L1']
    L2_features = ['cat1_L2', 'cat2_L2', 'xgb1_L2', 'xgb1_L2']

    # skriv ut cat_features till fil bevara å, ä, ö
    with open(pref+'CAT_RAW.txt', 'w', encoding='utf-8') as f:   # Måste editeras för att ta bort ev onödiga features
        for item in cat_features:
            f.write("%s\n" % item)

    # skriv ut num_features till fil
    with open(pref+'NUM_RAW.txt', 'w',  encoding='utf-8') as f:  # Måste editeras för att ta bort onödiga features
        for item in num_features:
            f.write("%s\n" % item)   
            
    # skriv ut META_features till fil
    with open(pref+'L1_OUTP_FEATURES.txt', 'w',  encoding='utf-8') as f:  # Adderas till den totala listan av features
        for item in L1_features:
            f.write("%s\n" % item)    
    with open(pref+'L2_OUTP_FEATURES.txt', 'w',  encoding='utf-8') as f:  # Används till medelvärdet för den slutliga prediktionen
        for item in L2_features:
            f.write("%s\n" % item)    


Exempel på hur feature-filerna skall läsas in

In [4]:

# läs in CAT_FEATURES.txt till cat_features
with open(pref+'CAT_FEATURES.txt', 'r', encoding='utf-8') as f:
    cat_features = f.read().split()        
    
# läs in NUM_FEATURES.txt till num_features
with open(pref+'NUM_FEATURES.txt', 'r', encoding='utf-8') as f:
    num_features = f.read().split()    

# läs in L1_OUTP_FEATURES.txt till L1_features
with open(pref+'L1_OUTP_FEATURES.txt', 'r', encoding='utf-8') as f:
    L1_features = f.read().split()
# läs in L2_OUTP_FEATURES.txt till L2_features
with open(pref+'L2_OUTP_FEATURES.txt', 'r', encoding='utf-8') as f:
    L2_features = f.read().split()
                    

In [5]:
L2_features


['cat1_L2', 'cat2_L2', 'xgb1_L2', 'xgb1_L2']

In [6]:

import xgboost as xgb
# TODO Testa andra encoders
# TODO Ta med samtliga kolumner hela tiden och använd X[USE_KOLUMNS] för fit() och predict()
# from sklearn.preprocessing import OneHotEncoder
import json

encoder_type = 'catboost'  # 'target', 'catboost' or 'catcodes'

# get travdata
v75 = td.v75(pref='../')
if encoder_type == 'target':
    df, _ = v75.förbered_data(extra=True, target_encode_list=cat_features)
elif encoder_type == 'catboost':
    print('Alla cat_features', cat_features)
    df, ENC = v75.förbered_data(extra=True, catboost_encode_list=cat_features)
    print('ENC.get_feature_names Ett',ENC.get_feature_names())
elif encoder_type == 'catcodes':    
    df,_ = v75.förbered_data(extra=True) 
else:
    raise ValueError('encoder_type must be target, catboost or catcodes')
name='xgb1L1'

print('ENC.get_feature_names',ENC.get_feature_names())

    # Läs in parametrar från fil
with open(pref+'optimera/params_'+name+'.json', 'rb') as f:
    params = json.load(f)
    params = params['params']

iterations = params['iterations'] if 'iterations' in params else 500
params.pop('iterations')  # Ta bort iterations från params

print('ung antal lopp',np.mean(df.avd.value_counts())*0.7)

# df.drop(['datum', 'avd', 'startnr'], axis=1, inplace=True)

# df = df[['bana','häst','streck','spår','dist','lopp_dist','start','ålder','kön','pris','y']].copy()
df.y = df.y.astype('float')

n_train = int(len(df) * 0.7)
X_train = df.iloc[:n_train].drop('y', axis=1)
y_train = df.iloc[:n_train]['y']
X_test = df.iloc[n_train:].drop('y', axis=1)
y_test = df.iloc[n_train:]['y']

# bool_features = list(X_train.select_dtypes(include=['bool']).columns)
# X_train[bool_features] = X_train[bool_features].astype('int')
# X_test[bool_features] = X_test[bool_features].astype('int')

# display(df.shape, len(cat_features)+len(num_features))
# display(df.info() )

if   encoder_type=='catcodes': # Använd category codes som värden 
    print('# Använd category codes')
    X_train_use = X_train.copy()
    X_test_use = X_test.copy()
    X_train_use[cat_features] = X_train[cat_features].astype('category')
    X_test_use[cat_features] = X_test[cat_features].astype('category')
    X_train_use[cat_features] = X_train_use[cat_features].apply(
        lambda x: x.cat.codes)
    X_test_use[cat_features] = X_test_use[cat_features].apply(
        lambda x: x.cat.codes)
elif encoder_type=='target' :   # Använd Target encoding 
    print('# TargetEncoder Redan fixat i förbered_data()')
    X_train_use = X_train.copy()
    X_test_use = X_test.copy()
elif encoder_type=='catboost' :   # Använd CatBoost encoding 
    print('CatBoostEncoder Redan fixat i förbered_data()')
    X_train_use = X_train.copy()
    X_test_use = X_test.copy()
else:
    raise ValueError('encoder_type måste vara "target" eller "catcodes"')    

display(X_train_use.head(2))
model = xgb.XGBClassifier(**params,
                        scale_pos_weight=12)



../all_data.csv
Loading dataframe from the file: ../all_data.csv
Alla cat_features ['bana', 'häst', 'kusk', 'kön', 'h1_kusk', 'h1_bana', 'h2_kusk', 'h2_bana', 'h3_kusk', 'h3_bana', 'h4_kusk', 'h4_bana', 'h5_kusk', 'h5_bana']
plac finns i df
Creating new catboost encoder
CatBoost encoding done
ENC.get_feature_names Ett ['bana', 'häst', 'kusk', 'kön', 'h1_kusk', 'h1_bana', 'h2_kusk', 'h2_bana', 'h3_kusk', 'h3_bana', 'h4_kusk', 'h4_bana', 'h5_kusk', 'h5_bana']
ENC.get_feature_names ['bana', 'häst', 'kusk', 'kön', 'h1_kusk', 'h1_bana', 'h2_kusk', 'h2_bana', 'h3_kusk', 'h3_bana', 'h4_kusk', 'h4_bana', 'h5_kusk', 'h5_bana']
ung antal lopp 4694.999999999999
CatBoostEncoder Redan fixat i förbered_data()


,datum,avd,bana,häst,kusk,streck,spår,dist,lopp_dist,start,ålder,kön,pris,h1_kusk,h1_bana,h1_spår,h1_plac,h1_pris,h1_odds,h1_kmtid,h2_kusk,h2_bana,h2_spår,h2_plac,h2_pris,h2_odds,h2_kmtid,h3_kusk,h3_bana,h3_spår,h3_plac,h3_pris,h3_odds,h3_kmtid,h4_kusk,h4_bana,h4_spår,h4_plac,h4_pris,h4_odds,h4_kmtid,h5_kusk,h5_bana,h5_spår,h5_plac,h5_pris,h5_odds,h5_kmtid,h1_dist,h2_dist,h3_dist,h4_dist,h5_dist,h1_auto,h2_auto,h3_auto,h4_auto,h5_auto,h1_perf,h2_perf,h3_perf,h4_perf,h5_perf,senast,delta1,delta2,delta3,delta4,startnr,rel_kr,streck_avst,rel_rank,h1_samma_bana,h2_samma_bana,h3_samma_bana,h1_samma_kusk,h2_samma_kusk,h3_samma_kusk,häst_namn,kusk_namn
0,2014-12-28,1.0,0.086178,0.110221,0.007834,5.0,6.0,2100.0,2100.0,0,6,0.080678,125000.0,0.083552,0.088300,3.0,2.0,35.0,3.92,16.8,0.104309,0.092676,3.0,1.0,30.0,3.70,14.9,0.090703,0.075912,3.0,15.0,125.0,52.42,14.3,0.035038,0.103769,3.0,15.0,70.0,5.20,13.9,0.028726,0.095196,3.0,15.0,25.0,2.2,12.3,2140.0,2140.0,2640.0,2140.0,1609.0,1,1,1,1,1,3935.030968,6006.507182,11.180340,8.366600,5.000000,21.0,19.0,17.0,10.0,18.0,0.0,0.102926,43.0,0.416667,False,False,False,True,True,True,allaballakaitoz,carl-erik lindblom
1,2014-12-28,1.0,0.086178,0.099342,0.098834,7.0,12.0,2100.0,2100.0,0,7,0.080678,125000.0,0.149669,0.101346,7.0,4.0,125.0,12.45,30.0,0.145868,0.081085,7.0,5.0,100.0,7.55,30.0,0.106469,0.099475,7.0,6.0,70.0,15.58,13.6,0.101773,0.074843,7.0,6.0,70.0,2.55,15.1,0.095652,0.103340,7.0,6.0,100.0,7.2,15.6,1640.0,3180.0,2140.0,2640.0,3180.0,1,0,1,1,0,2735.738970,1484.131591,753.137355,753.137355,900.171313,8.0,7.0,24.0,14.0,11.0,0.0,0.114914,41.0,0.250000,False,False,False,False,False,True,aristocat boko,ulf ohlsson


In [ ]:

# Använd encoder.fit_transform() för att omvandla kategoriska variabler i df_train
# if ohe:
#     X_train_use = encoder.fit_transform(X_train)
# else:
#     X_train_use = X_train    
    
use_features = cat_features + num_features
print(f'tränar på X_train_use med shape {X_train_use[use_features].shape}')    
model.fit(X_train_use[use_features], y_train, verbose=True)



In [ ]:
# Beräkna accuracy
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

lim=0.5   #0.57

use_features = cat_features + num_features    
print(f'predict på X_test_use med shape {X_test_use[use_features].shape}')        
y_pred = (model.predict_proba(X_test_use[use_features])[:, 1] > lim)
y_pred = y_pred.astype(int)
print('Antal tippade', np.sum(y_pred), 'av', len(y_pred),   '(', np.sum(y_pred)/len(y_pred)*12, ')')
# Beräkna konfusionsmatris
confusion_matrix = confusion_matrix(y_test, y_pred, normalize=None)

# Rita upp värme kartan med den normaliserade matrisen
# print(f'Confusion matrix: \n{confusion_matrix}')

import seaborn as sns
# Skapa en värmevägskarta av konfusionsmatrisen
sns.heatmap(confusion_matrix, annot=True, fmt='d',
            xticklabels=['predicted negative', 'predicted positive'],
            yticklabels=['true negative', 'true positive'])

# Beräkna precision och recall
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# print(f'Precision: {precision:.2f}')
# print(f'Recall: {recall:.2f}')

# from sklearn.metrics import f1_score
# # Beräkna F1-score
# f1 = f1_score(y_test, y_pred)
# print(f'F1-score: {f1:.2f}')

from sklearn.metrics import roc_auc_score
# Mät AUC för y_pred
auc = roc_auc_score(y_test, y_pred)

# Skriv ut AUC
print(f'AUC: {auc:.2f}')


# Skriv ut en rapport med AUC och andra prestandametriker
print(classification_report(y_test, y_pred))


### Använder Category Codes
tot=9014+3862+274+935  
935/tot*100

AUC: 0.74   
_________________precision______recall______f1-score____support  

           0       0.97      0.70      0.81     12876
           1       0.19      0.77      0.31      1209 

    accuracy                           0.71     14085  
   macro avg_______0.58__________0.74_________0.56_________14085  
weighted avg____0.90__________0.71_________0.77_________14085  

In [ ]:
((753/(753+2194))*2.5)

In [ ]:
y_counts = df.y.value_counts(normalize=True)

# Skriv ut antalet 1:or och 0:or
print(y_counts*11.61)

## Kollar stacking i sklearn

In [ ]:
# from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# import LightGBM classifier
from lightgbm import LGBMClassifier
# import Ridge classifier
from sklearn.linear_model import RidgeClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import TimeSeriesSplit

ts = TimeSeriesSplit(n_splits=3)
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=2022)),
    ('lg', LGBMClassifier(n_estimators=100, random_state=2022))
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=RidgeClassifier(),
    stack_method='predict_proba',
)

v75 = td.v75(pref='../')
target_encode_list = ['häst', 'kön','bana', 'kusk', 'h1_kusk', 'h2_kusk', 'h3_kusk',
                       'h4_kusk', 'h5_kusk', 'h1_bana', 'h2_bana', 'h3_bana', 'h4_bana', 'h5_bana']
df,_ = v75.förbered_data(missing_num=True, missing_cat=True,
                      target_encode_list=target_encode_list)  # num hanteras av catboost

display(df)
for train_index, test_index in ts.split(df):
    print("TRAIN:", train_index, "TEST:", test_index)


In [ ]:
df.iloc[-1:]

# add a list to the row with datum=d
l=[None]*69
l[0]=69
d= "2022-10-01"

lr = pd.DataFrame([[d]+l], columns=df.columns) 
display(lr)
def add_row(df, d, l):
    df.loc[(df.datum==d ) & (df.spår==9),:] = [d] + l
    print([d]+l)
    return df
# df=add_row(df, d, l)
pd.concat([df,lr], ignore_index=True)

In [ ]:
    
for train_index, test_index in ts.split(df):
    train, test = df.iloc[train_index], df.iloc[test_index]
    print(clf.fit(train.drop(['datum','avd','y'], axis=1),train.y).score(test.drop(['datum','avd','y'], axis=1), test.y))

df.info()

In [ ]:
# skapa modeller
#           name, ant_hästar, proba, kelly, motst_ant, motst_diff,  ant_favoriter, only_clear, streck
typ6 = tp.Typ('typ6', True,       True, False,     0,     False,          0,            False,    True,  pref)
typ1 = tp.Typ('typ1', False,      True, False,     2,     True,           2,            True,     False, pref)
typ9 = tp.Typ('typ9', True,       True, True,      2,     True,           2,            True,     True,  pref)
# typ16 = tp.Typ('typ16', True,      True, True,      2,    True,           2,            False,    True,  pref)

typer = [typ6, typ1, typ9]    # , typ16]  # load a file with pickl


# with open(pref+'modeller\\meta_ridge_model.model', 'rb') as f:
#     meta_model = pickle.load(f)


In [ ]:
def remove_features(df_, remove_mer=[]):
    df = df_.copy()
    df.drop(['startnr', 'vodds', 'podds', 'bins', 'h1_dat',
            'h2_dat', 'h3_dat', 'h4_dat', 'h5_dat'], axis=1, inplace=True)
    if remove_mer:
        df.drop(remove_mer, axis=1, inplace=True)

    return df


In [ ]:
# Kelly-värde baserat på streck omvandlat till odds
def kelly(proba, streck, odds):  # proba = prob winning, streck i % = streck
    with open(pref+'rf_streck_odds.pkl', 'rb') as f:
        rf = pickle.load(f)

    if odds is None:
        o = rf.predict(streck.copy())
    else:
        o = rf.predict(streck.copy())

    # for each values > 40 in odds set to 1
    o[o > 40] = 1
    return (o*proba - (1-proba))/o


In [ ]:
###############################################
#              LEARNING                       #
###############################################

def skapa_stack_learning(X_, y, save=True):
    X = X_.copy()
    stacked_data = pd.DataFrame()
    for typ in typer:
        nr = typ.name[3:]
        stacked_data['proba'+nr] = typ.predict(X)
        stacked_data['kelly' + nr] = kelly(stacked_data['proba' + nr], X[['streck']], None)

    # print(stacked_data.columns)
    assert len(stacked_data) == len(y), f'stacked_data {len(stacked_data)} and y {len(y)} should have same length'
    
    return stacked_data, y   # enbart stack-info
    
##### KNN (meta model) #####  
def learn_meta_knn_model(X, y, algorithm = 'auto', leaf_size = 1, metric = 'chebyshev', n_neighbors = 600, p = 1, weights = 'distance', save = True):
    from sklearn.neighbors import KNeighborsClassifier

    knn_model = KNeighborsClassifier( p=p, n_neighbors=n_neighbors, metric=metric, leaf_size=leaf_size, algorithm=algorithm, weights=weights, n_jobs=6)
    knn_model.fit(X, y)
    # pickle save stacking
    if save:
        with open(pref+'modeller/meta_knn_model.model', 'wb') as f:
            pickle.dump(knn_model, f)

    return knn_model

##### RidgeClassifier (meta model) #####
def learn_meta_ridge_model(X, y, alpha=1, class_weight='balanced', save=True):
    from sklearn.linear_model import RidgeClassifier

    ridge_model = RidgeClassifier(alpha=alpha,class_weight=class_weight, random_state=2022)
    ridge_model.fit(X, y)
    # pickle save stacking
    if save:
        with open(pref+'modeller/meta_ridge_model.model', 'wb') as f:
            pickle.dump(ridge_model, f)

    return ridge_model

##### RandomForestClassifier (meta model) #####
def learn_meta_rf_model(X, y, n_estimators=100, max_depth=None, class_weight='balanced', save=True):
    from sklearn.ensemble import RandomForestClassifier

    rf_model = RandomForestClassifier(n_estimators = n_estimators, max_depth=max_depth, n_jobs=6, class_weight=class_weight, random_state=2022)
    rf_model.fit(X, y)
    # pickle save stacking
    if save:
        with open(pref+'modeller/meta_rf_model.model', 'wb') as f:
            pickle.dump(rf_model, f)

    return rf_model

##### LassoClassifier (meta model) #####
def learn_meta_lasso_model(X, y, alpha=1, max_iter=1000,  save=True):
    from sklearn.linear_model import Lasso

    lasso_model = Lasso(alpha=alpha, max_iter=max_iter, random_state=2022)
    lasso_model.fit(X, y)
    # pickle save stacking
    if save:
        with open(pref+'modeller/meta_lasso_model.model', 'wb') as f:
            pickle.dump(lasso_model, f)

    return lasso_model


def learn_meta_model(X, y,  meta='ridge', alpha=1,n_estimators=100, max_iter=1000, max_depth=None, class_weight='balanced', save=True):
    if meta == 'ridge':
        return learn_meta_ridge_model(X, y, alpha=alpha, class_weight=class_weight, save=save)
    elif meta == 'rf':
        return learn_meta_rf_model(X, y, n_estimators=n_estimators, max_depth=max_depth, class_weight=class_weight, save=save)
    elif meta == 'lasso':
        return learn_meta_lasso_model(X, y, alpha=alpha, max_iter=max_iter, save=save)
    elif meta == 'knn':
        return learn_meta_knn_model(X, y, save=save)
    else:
        assert False, f'{meta} is not a valid meta model'


In [ ]:
def förbered(df,meta_fraction):
    # Följande datum saknar avd==5 och kan inte användas
    saknas = ['2015-08-15', '2016-08-13', '2017-08-12']
    df = df[~df.datum.isin(saknas)]
    X = df.copy()
    X.drop('plac', axis=1, inplace=True)
    
    # läs in FEATURES.txt
    with open(pref+'FEATURES.txt', 'r',encoding='utf-8') as f:    
        features = f.read().splitlines()
     
    X=X[features]
    
    assert len(features) == len(X.columns), f'features {len(features)} and X.columns {len(X.columns)} are not the same length'   
    assert set(features) == set(X.columns), f'features {set(features)} and X.columns {set(X.columns)} are not the same'
    
    y = (df.plac == 1)*1   # plac 1 eller 0

    for f in ['häst', 'bana', 'kusk', 'h1_kusk', 'h2_kusk', 'h3_kusk', 'h4_kusk', 'h5_kusk', 'h1_bana', 'h2_bana', 'h3_bana', 'h4_bana', 'h5_bana']:
        X[f] = X[f].str.lower()

    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)
    if meta_fraction==0: 
        # no meta data created
        return X,y,None,None
    
    # use a fraction for meta data
    meta_antal = int(len(X.datum.unique())*meta_fraction)
    meta_datum = X.datum.unique()[-meta_antal:]

    X_val = X.loc[X.datum.isin(meta_datum)]
    y_val = y[X_val.index]
    X=X.loc[~X.datum.isin(meta_datum)]
    y=y.loc[X.index]
    return X, y, X_val, y_val


In [ ]:

def TimeSeries_learning( typer, n_splits=5,meta_fraction=0.2, meta='rf', save=True, learn_models=True):
    """
    Skapar en stack av X_test och y_test från alla typer. Används som input till meta_model.
        - learn_models=True betyder att vi både gör en learning och skapar en stack
        - learn_models=False betyder att vi bara skapar en stack och då har save ingen funktion
    """    
    
    df_all = pd.read_csv(pref+'all_data.csv')
             
    # print('sista datum',df_all.datum.iloc[-1])
    
    X, y, _, _ = förbered(df_all, meta_fraction=meta_fraction)
    print('shape of X', X.shape)
    print('sista datum', X.datum.iloc[-1], 'resp', df_all.datum.iloc[-1])
    
    ts = TimeSeriesSplit(n_splits=n_splits)
    stacked_data=pd.DataFrame(columns=['proba6', 'proba1', 'proba9',  'kelly6', 'kelly1', 'kelly9','y'])
        
    for enum,(train_index, test_index) in enumerate(ts.split(X,y)):
        print('\nshape of X_train', X.iloc[train_index].shape, 'shape of X_test', X.iloc[test_index].shape)
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]
        X_test = X.iloc[test_index]
        y_test = y.iloc[test_index]
        temp_df = pd.DataFrame()
        temp_df['y'] =  y_test
        print('enum',enum)
        for typ in typer:
            print('  ',typ.name)
            if learn_models:
                # eftersom predict(X_test) gör en load-model så gör vi save här
                cbc = typ.learn(X_train, y_train, X_test, y_test, save=save)
                print('  ','best_iteration', cbc.get_best_iteration())
                
            this_proba=typ.predict(X_test)
            
            nr = typ.name[3:]
            
            temp_df['proba'+nr] = this_proba

            this_kelly=kelly(this_proba, X_test[['streck']], None)
            temp_df['kelly' + nr] = this_kelly
        
        
        stacked_data = pd.concat([stacked_data, temp_df],ignore_index=True)
        stacked_data.y = stacked_data.y.astype(int)
        
    # step 2: learn models on all of X - what iteration to use?
    print('\nFull X learning')
    for typ in typer:
        print(typ.name)
        if learn_models:
            cbc = typ.learn(X, y, None, None, iterations=100, save=save)

    # step 3: learn meta model
    print('Null values', stacked_data.isnull().sum())
    meta_model = learn_meta_model(stacked_data.drop(['y'], axis=1), stacked_data['y'], alpha=0.001,
                              max_iter=55, n_estimators=360, max_depth=5, meta=meta, class_weight=None)

    print('✔️ Learning done')

    return stacked_data    


In [ ]:
##############################################################
#                     VALIDATE                               #
##############################################################

print('skall inte köras efter final learning - sparar modeller baserade på mindra data')

def predict_meta_ridge_model(X, ridge_model=None):
    if ridge_model is None:
        with open(pref+'modeller/meta_ridge_model.model', 'rb') as f:
            ridge_model = pickle.load(f)

    return ridge_model._predict_proba_lr(X)

def predict_meta_rf_model(X, rf_model=None):
    if rf_model is None:
        with open(pref+'modeller/meta_rf_model.model', 'rb') as f:
            rf_model = pickle.load(f)

    return rf_model.predict_proba(X)

def predict_meta_lasso_model(X, lasso_model=None):
    if lasso_model is None:
        with open(pref+'modeller/meta_lasso_model.model', 'rb') as f:
            lasso_model = pickle.load(f)

    return lasso_model.predict(X)

def predict_meta_knn_model(X, knn_model=None):
    if knn_model is None:
        print('ladda in knn_model')
        with open(pref+'modeller/meta_knn_model.model', 'rb') as f:
            knn_model = pickle.load(f)
    print('predict knn')
    return knn_model.predict_proba(X)

def predict_meta_model(X, meta_model=None):
    if meta_model == 'ridge':
        return predict_meta_ridge_model(X, meta_model)[:, 1]
    elif meta_model == 'rf':
        return predict_meta_rf_model(X, meta_model)[:, 1]
    elif meta_model == 'lasso':
        return predict_meta_lasso_model(X, meta_model)
    elif meta_model == 'knn':
        return predict_meta_knn_model(X, meta_model)[:,1]
    elif meta_model == None:
        assert False, 'ingen meta_model angiven'
    else:             
        return meta_model.predict(X)

def display_scores(y_true, y_pred):
    print('AUC', roc_auc_score(y_true, y_pred), '  ')
    # and the F1 score
    print('F1', f1_score(y_true, y_pred), '  ')
    #accuracy
    print('Acc', accuracy_score(y_true, y_pred), '  ')


def plot_confusion_matrix(y_true, y_pred, typ, fr=0.1, to=0.5, step = 0.001):
    #### Först:  hitta ett treshold som tippar ca 2.5 hästar per avd ####
    for tresh in np.arange(fr, to, step):
        cost = 12*sum(y_pred > tresh)/len(y_pred)
        if cost < 2.5:
            break
    tresh = round(tresh, 4)
    # print(f'Treshold: {tresh}\n')
    y_pred = (y_pred > tresh).astype(int)
    # confusion_matrix_graph(y_true, y_pred, f'{typ} treshold={tresh}')
    
    #### Sedan: confusion matrix graph ####
    title = f'{typ} treshold={tresh}'
    cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
    fig, ax = plt.subplots()
    sns.set(font_scale=2.0)
    sns.heatmap(cm, annot=True, fmt=".2f", linewidths=.5, square=True, cmap='Blues_r')

    # increase font size
    plt.rcParams['font.size'] = 20
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title(title)
    # plot fig
    plt.show()
    
    #### print scores ####
    display_scores(y_true, y_pred)
    print('spelade per lopp:', 12 * sum(y_pred)/len(y_pred))
    

def validate(meta_model=None, drop=[]):
    print('Only accurate directly after "Learn TimeSeries"')
    df_all = pd.read_csv(pref+'all_data.csv')
    print('sista datum', df_all.datum.iloc[-1])

    _, _, X_val, y_val = förbered(df_all, meta_fraction=0.2)
   
    # create the stack from validation data
    stacked_meta_val, y_val = skapa_stack_learning(X_val, y_val)
    stacked_meta_val = stacked_meta_val.drop(drop, axis=1)
    stacked_meta_val['meta'] = predict_meta_model(stacked_meta_val, meta_model=meta_model)
    stacked_meta_val['y'] = y_val.values
    stacked_meta_val['avd'] = X_val.avd.values
    
    ##############################################################
    #                          Meta model                        #
    ##############################################################
    y_true = stacked_meta_val['y']
    y_pred = stacked_meta_val['meta']
    display(y_pred)
    plot_confusion_matrix(y_true, y_pred, 'meta', fr=0.0, to=0.1, step = 0.0001)
    
    ################################################################
    #                         proba 6, 1, 9                        #
    ################################################################
    for typ in typer:
        name = 'proba' + typ.name[3:]
        y_pred = stacked_meta_val[name]
        plot_confusion_matrix(y_true, y_pred, name)


## Final learning

In [ ]:
##############################################################
#            FINAL LEARNING steps                            #
##############################################################
def final_learning(typer, n_splits=5, learn_models=True):
    # step 1: learn models and produce the stacked data
    print('Step 1: Final learn models and produce the stacked data')
    stacked_data = TimeSeries_learning(typer, n_splits=n_splits, meta_fraction=0, save=True, learn_models=learn_models)

    # step 2: learn meta model
    meta = 'knn'
    print('Step 2: Final learn meta model', meta)
    
    meta_model = learn_meta_model(stacked_data.drop(['y'], axis=1), stacked_data['y'], alpha=0.001, max_iter=55, n_estimators=360, max_depth=5, meta=meta, class_weight=None)

    print(meta_model)
    
    if meta != 'knn':
        # print(meta_model.coef_)
        # l = list(zip(stacked_data.columns, np.round(meta_model.coef_, 22)))

        l = list(zip(stacked_data.columns, np.round(meta_model.feature_importances_, 4)))
        l.sort(key=lambda x: x[1], reverse=True)
        print('feature imp:', l)
        
    print('✔️ Final learning done')
    return stacked_data


# Run parts of the program

Timeseries learning - skapar stacked data

In [ ]:
stacked_data=TimeSeries_learning(typer, n_splits=3, meta_fraction=0.2, meta='knn', save=True, learn_models=True)

Learn meta model on stacked_data

In [ ]:
meta_model = learn_meta_model(stacked_data.drop(['y'], axis=1), stacked_data['y'], alpha=47, max_iter=40, n_estimators=360, max_depth=5, meta='knn', class_weight=None)
print(meta_model)
# print(meta_model.coef_)
# l = list(zip(stacked_data.columns, np.round(meta_model.coef_, 22)))

# l = list(zip(stacked_data.columns, np.round(meta_model.feature_importances_, 4)))
# l.sort(key=lambda x: x[1], reverse=True)
# print('feature imp:', l)


Validate meta model and typ models

In [ ]:
validate( meta_model=meta_model, drop=[])  

Final Learning

In [ ]:
final_stacked_data = final_learning(typer, n_splits=5, learn_models=True)


In [ ]:
meta='knn'
meta_model = learn_meta_model(final_stacked_data.drop(['y'], 
                              axis=1), final_stacked_data['y'], alpha=0.001, max_iter=55, n_estimators=360, max_depth=5, meta='knn', class_weight=None)
print(meta_model)

if meta=='knn':
    pass 
else:
    # print(meta_model.coef_)
    # l = list(zip(final_stacked_data.columns, np.round(meta_model.coef_, 22)))

    l = list(zip(stacked_data.columns, np.round(meta_model.feature_importances_, 4)))
    l.sort(key=lambda x: x[1], reverse=True)
    print('feature imp:', l)

# Check out meta models

In [ ]:
###############################################################################
#                  optimize models                                            #
###############################################################################


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
# 0.814077	{'algorithm': 'auto', 'leaf_size': 1, 'metric': 'chebyshev', 'n_neighbors': 1000, 'p': 1, 'weights': 'distance'}
# 0.814750	{'algorithm': 'auto', 'leaf_size': 1, 'metric': 'chebyshev', 'n_neighbors': 2000, 'p': 1, 'weights': 'distance'}
# uppdatera ovan och i app_learn

model_params = {
    'knn': {
        'model': KNeighborsClassifier(n_jobs=6),
        'params': {
            'algorithm': ['auto'],
            'leaf_size': [1, 5, 30],
            'metric': ['euclidean', 'manhattan', 'chebyshev'],
            'n_neighbors': [5, 75, 130, 500, 600, 700,1000, 1500, 2000, 4000],
            'p': [1, 2],
            'weights': ['uniform', 'distance'],
        }
    },
    'lasso': {
        'model': Lasso(random_state=2022),
        'params': {
            'alpha': [0.00001, 0.00002, 0.00003, 0.0001, 0.0002, 0.0005],
            'max_iter': [55, 30, 40, 50, 55,70,100,500,1000],
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(n_jobs=4, random_state=2022),
        'params': {
            'n_estimators': [300, 366, 400, 500],
            'max_depth': [ 5, 6, 8],
            'class_weight': [None],
        }
    },
    'ridge': {  
        'model': RidgeClassifier(random_state=2022),
        'params': {
            'alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 100],
            'class_weight': [None],
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            'C': [1, 5, 10]
        }
    },
    'naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {'priors': [None]}
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 3, 4, 5,6],

        }
    }
}


In [ ]:
from sklearn.model_selection import GridSearchCV
tscv = TimeSeriesSplit(n_splits=5)
# final_stacked_data = final_learning(typer, n_splits=5, learn_models=True)

display(pd.read_csv('grid_search_meta_results.csv').sort_values(by='best_score', ascending=False))
scores = []

for model_name, mp in model_params.items():
    print(model_name,end=', ')
    clf = GridSearchCV(mp['model'], mp['params'], n_jobs=4, 
                       cv=tscv.split(final_stacked_data.drop(['y'], axis=1)), 
                       scoring='roc_auc',  
                       return_train_score=False
                      )
    
    clf.fit(final_stacked_data.drop(['y'],axis=1), final_stacked_data['y'])
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    print('best_score =', clf.best_score_, clf.best_params_)
df_grid = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params']).sort_values(by='best_score', ascending=False)
df_grid.to_csv('grid_search_meta_results.csv', index=False)
df_grid


In [ ]:
display(pd.read_csv('grid_search_meta_results.csv').sort_values(by='best_score', ascending=False))
